In [ ]:
!pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
path = '/content/drive/MyDrive/diabetes_clean.csv'

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df = pd.read_csv(path)
df.drop(columns = 'Unnamed: 0', inplace=True)
df

,State,GenHlth,NoDocCost,KidneyDisease,Married,Race,HasInsurance,PhysHlthBad,MentHelthBad,Exercise,...,IncomeLvl,SmokerLvl,HeavyDrinker,FruitJuiceIntake,FruitIntake,DrkGrnVegIntake,FrenchFryIntake,PotatoIntake,OtherVegIntake,DiabetesStatus
0,1.0,2.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,100.0,0.0,14.0,14.0,43.0,1.0
1,1.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,...,5.0,0.0,0.0,71.0,43.0,10.0,57.0,27.0,71.0,1.0
2,1.0,5.0,0.0,0.0,1.0,6.0,1.0,2.0,0.0,1.0,...,2.0,0.0,0.0,0.0,100.0,100.0,29.0,29.0,100.0,1.0
3,1.0,4.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,5.0,0.0,0.0,7.0,7.0,29.0,100.0,7.0,100.0,1.0
4,1.0,4.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,57.0,43.0,29.0,10.0,200.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139668,20.0,3.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,...,4.0,3.0,0.0,7.0,29.0,14.0,3.0,14.0,14.0,3.0
139669,15.0,4.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,1.0,...,1.0,3.0,0.0,0.0,2.0,0.0,3.0,2.0,7.0,3.0
139670,48.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,...,7.0,0.0,0.0,0.0,100.0,43.0,3.0,13.0,200.0,3.0
139671,35.0,2.0,0.0,0.0,1.0,4.0,1.0,0.0,1.0,1.0,...,3.0,1.0,0.0,13.0,17.0,13.0,33.0,33.0,23.0,3.0


In [ ]:
X, y = df.iloc[:,:26], df.DiabetesStatus
y

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
139668    3.0
139669    3.0
139670    3.0
139671    3.0
139672    3.0
Name: DiabetesStatus, Length: 139673, dtype: float64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=24)

In [ ]:
over = SMOTENC(categorical_features=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,19])
under = RandomUnderSampler()

In [ ]:
xres, yres = over.fit_resample(X_train, y_train)

In [ ]:
yres.value_counts()

3.0    80041
1.0    80041
4.0    80041
Name: DiabetesStatus, dtype: int64

In [ ]:
y_train.value_counts()

3.0    80041
1.0    27110
4.0     4587
Name: DiabetesStatus, dtype: int64

In [ ]:
xres2, yres2 = under.fit_resample(xres, yres)

In [ ]:
yres2.value_counts()

1.0    80041
3.0    80041
4.0    80041
Name: DiabetesStatus, dtype: int64

In [ ]:
enc=OneHotEncoder()
enc_data=pd.DataFrame(enc.fit_transform(xres2[['Race']]).toarray())
New_xres2=xres2.join(enc_data)
New_xres2=New_xres2.rename(columns = {0:'Race_White',
                            1:'Race_Black',
                            2:'Race_Asian',
                            3:'Race_Native',
                            4:'Race_Hisp',
                            5:'Race_Other'})
New_xres2 = New_xres2.drop('Race',axis=1)

In [ ]:
New_X_test = pd.get_dummies(X_test['Race'])
New_X_test=New_X_test.rename(columns = {1:'Race_White',
                            2:'Race_Black',
                            3:'Race_Asian',
                            4:'Race_Native',
                            5:'Race_Hisp',
                            6:'Race_Other'})
New_X_test=X_test.join(New_X_test)
New_X_test = New_X_test.drop('Race',axis=1)

In [ ]:
temp = pd.concat([New_xres2, yres2], axis=1)
temp_test = pd.concat([New_X_test, y_test], axis = 1)

In [ ]:
temp.iloc[:,1:].to_csv('train.csv')
temp_test.iloc[:,1:].to_csv('test.csv')